# Fund24. Embedding, Recurrent Layer
- 희소 표현(Sparse Representation) : 벡터의 특정 차원에 단어 혹은 의미를 직접 매핑하는 방식
- 분포 가설(Distribution Hypothesis) : 유사한 맥락에서 나타나는 단어는 그 의미도 비슷하다라고 가정하는 것. 
- 분산 표현(Distributed Representation) : 유사한 맥락에 나타난 단어들끼리는 두 단어 벡터 사이의 거리를 가깝게 하고, 그렇지 않은 단어들끼리는 멀어지도록 조금씩 조정해 이런 방식으로 얻어지는 단어 벡터를 말한다. 벡터의 특정 차원이 특정 의미를 담고 있는 것이 아니라 의미가 벡터의 여러 차원에 분산되어 있으리라고 여기게 된다.  
> 분산 표현을 사용하며 희소 표현과는 달리 단어 간 유사도를 계산으로 구할 수 있다는 장점이 있다.  

- **Embedding 레이어**는 바로 단어의 분산 표현을 구현하기 위한 레이어다. 간단하게 컴퓨터용 단어 사전이라고 할 수 있다. 
- 이러한 분산 표현은 단어를 표현할 때만 사용되는 것이 아니라, 의미적 유사성을 가지는 여러 가지 것들을 컴퓨터에게 가르치는 방법으로 두루 활용된다. 

## Embedding 레이어
- n개의 단어를 k개의 차원(Embedding Size)으로 표현해달라고 하면 Weight는 자연스럽게 단어의 개수, Embedding 사이즈로 정의된다. 
- Embedding 레이어는 입력으로 들어온 단어를 분산 표현으로 연결해주는데, 이는 Weight에서 특정 행을 읽어오는 것과 같아 이 레이어를 룩업 테이블이라고도 한다. 

In [1]:
import tensorflow as tf

vocab = {      # 사용할 단어 사전 정의
    "i": 0,
    "need": 1,
    "some": 2,
    "more": 3,
    "coffee": 4,
    "cake": 5,
    "cat": 6,
    "dog": 7
}

sentence = "i i i i need some more coffee coffee coffee"
# 위 sentence
_input = [vocab[w] for w in sentence.split()]

vocab_size = len(vocab)

one_hot = tf.one_hot(_input, vocab_size)
print(one_hot.numpy())  # 원-핫 인코딩 벡터 출력

[[1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]]


In [2]:
# 생성된 원-핫 벡터를 Linear 레이어에 넣어보자. 

distribution_size = 2  # 보기 좋게 2차원으로 분산 표현해보자. 
linear = tf.keras.layers.Dense(units=distribution_size, use_bias=False)
one_hot_linear = linear(one_hot)

print("Linear Weight")
print(linear.weights[0].numpy)

print("\nOne-Hot Linear Result")
print(one_hot_linear.numpy())

Linear Weight
<bound method BaseResourceVariable.numpy of <tf.Variable 'dense/kernel:0' shape=(8, 2) dtype=float32, numpy=
array([[-0.7337723 ,  0.39723778],
       [ 0.64453375,  0.06135106],
       [-0.03493506,  0.05769604],
       [-0.6283457 , -0.6865303 ],
       [-0.6610942 ,  0.12432814],
       [-0.1404286 , -0.10800207],
       [-0.3498476 ,  0.31942296],
       [-0.20574558,  0.6225661 ]], dtype=float32)>>

One-Hot Linear Result
[[-0.7337723   0.39723778]
 [-0.7337723   0.39723778]
 [-0.7337723   0.39723778]
 [-0.7337723   0.39723778]
 [ 0.64453375  0.06135106]
 [-0.03493506  0.05769604]
 [-0.6283457  -0.6865303 ]
 [-0.6610942   0.12432814]
 [-0.6610942   0.12432814]
 [-0.6610942   0.12432814]]


- 원-핫 벡터에 Linear 레이어를 적용하니 Linear 레이어의 Weight에서 단어 인덱스 배열 `[0, 0, 0, 0, 1, 2, 3, 4, 4, 4]`에 해당하는 행만 읽어왔다. 
- 원 핫 인코딩을 위한 단어 사전을 구축하고 단어를 사전의 인덱스로 변환만 해주면 Embedding 레이어를 완벽하게 사용할 수 있다는 것이다. 

In [3]:
some_words = tf.constant([[3, 57, 35]])
# 3번 단어 / 57번 단어 / 35번 단어로 이루어진 한 문장

print("Embedding을 진행할 문장: ", some_words.shape)
embedding_layer = tf.keras.layers.Embedding(input_dim=64, output_dim=100)
# 총 64개의 단어를 포함한 Embedding 레이어를 선언
# 각 단어는 100차원으로 분산 표현

print("Embedding된 문장: ", embedding_layer(some_words).shape)
print("Embedding Layer의 Weight 형태: ", embedding_layer.weights[0].shape)

Embedding을 진행할 문장:  (1, 3)
Embedding된 문장:  (1, 3, 100)
Embedding Layer의 Weight 형태:  (64, 100)


- 기본적으로 딥러닝은 미분을 기반으로 동작하는데, Embedding 레이어는 그저 단어를 대응시켜 줄 뿐이니 미분이 불가능하다. 
- 따라서 신경망 설계를 할 때, **어떤 연산 결과를 Embedding 레이어에 연결시키는 것은 불가능**하다.
- Embedding 레이어는 **입력에 직접 연결되게** 사용해야 한다는 것을 꼭 기억하자. 

## Recurrent 레이어 (1) RNN
- 문장이나 영상, 음성 등의 데이터는 한 장의 이미지 데이터와는 달리 **순차적인(Sequential) 특성**을 가진다. 
- 이런 순차 데이터를 처리하기 위해 고안된 것이 RNN(Recurrent Neural Network)다. 
- `(입력의 차원, 출력의 차원)`에 해당하는 **단 하나의 Weight를 순차적으로 업데이트**하는 것이 RNN이다.
- 한 문장을 읽고 처리하는데 여러 번의 연산이 필요해 다른 레이어에 비해 느리다. 
- RNN의 고질적인 문제점으로, 입력의 앞부분이 뒤로 갈수록 옅어져 손실이 발생한다. **(Vanishing Gradient)**

In [4]:
sentence = "What time is it ?"
dic = {
    "is": 0,
    "it": 1,
    "What": 2,
    "time": 3,
    "?": 4
}

print("RNN에 입력할 문장: ", sentence)

sentence_tensor = tf.constant([[dic[word] for word in sentence.split()]])

print("Embedding을 위한 단어 매핑: ", sentence_tensor.numpy())
print("입력 문장 데이터 형태: ", sentence_tensor.shape)

embedding_layer = tf.keras.layers.Embedding(input_dim=len(dic), output_dim=100)
emb_out = embedding_layer(sentence_tensor)

print("\nEmbedding 결과: ", emb_out.shape)
print("Embedding Layer의 Weight 형태: ", embedding_layer.weights[0].shape)

rnn_seq_layer = \
tf.keras.layers.SimpleRNN(units=64, return_sequences=True, use_bias=False)
rnn_seq_out = rnn_seq_layer(emb_out)

print("\nRNN 결과 (모든 Step Output): ", rnn_seq_out.shape)
print("RNN Layer의 Weight 형태: ", rnn_seq_layer.weights[0].shape)

rnn_fin_layer = tf.keras.layers.SimpleRNN(units=64, use_bias=False)
rnn_fin_out = rnn_fin_layer(emb_out)

print("\nRNN 결과 (최종 Step Output): ", rnn_fin_out.shape)
print("RNN Layer의 Weight 형태: ", rnn_fin_layer.weights[0].shape)

RNN에 입력할 문장:  What time is it ?
Embedding을 위한 단어 매핑:  [[2 3 0 1 4]]
입력 문장 데이터 형태:  (1, 5)

Embedding 결과:  (1, 5, 100)
Embedding Layer의 Weight 형태:  (5, 100)

RNN 결과 (모든 Step Output):  (1, 5, 64)
RNN Layer의 Weight 형태:  (100, 64)

RNN 결과 (최종 Step Output):  (1, 64)
RNN Layer의 Weight 형태:  (100, 64)


- 어떤 문장이 긍정인지 부정인지 나누기 위해서라면 **문장을 모두 읽은 후, 최종 Step의 Output만 확인**해도 판단이 가능하다. 
- 하지만 문장을 생성하는 경우라면 **이전 단어를 입력으로 받아 생성된 모든 다음 단어, 즉 모든 Step에 대한 Output**이 필요하다. 
- 이것은 위 코드에서 `tf.keras.layers.SimpleRNN` 레이어의 `return_sequences` 인자로 조절할 수 있다. 

In [5]:
lstm_seq_layer = tf.keras.layers.LSTM(units=64, return_sequences=True, use_bias=False)
lstm_seq_out = lstm_seq_layer(emb_out)

print("\nLSTM 결과 (모든 Step Output): ", lstm_seq_out.shape)
print("LSTM Layer의 Weight 형태: ", lstm_seq_layer.weights[0].shape)

lstm_fin_layer = tf.keras.layers.LSTM(units=64, use_bias=False)
lstm_fin_out = lstm_fin_layer(emb_out)

print("\nLSTM 결과 (최종 Step Output):", lstm_fin_out.shape)
print("LSTM Layer의 Weight 형태:", lstm_fin_layer.weights[0].shape)


LSTM 결과 (모든 Step Output):  (1, 5, 64)
LSTM Layer의 Weight 형태:  (100, 256)

LSTM 결과 (최종 Step Output): (1, 64)
LSTM Layer의 Weight 형태: (100, 256)


- Embeddig 벡터의 차원수(unit)의 크기가 동일할 경우, Weight의 크기가 위에서 사용한 SimpleRNN의 4배나 된다.

## Recurrent 레이어 (2) LSTM
- LSTM(Long Short-Term Memory)의 약어로 **기울기 소실 문제를 해결하기 위해 고안된 RNN 레이어**다. 
- LSTM은 SimpleRNN보다 4배나 큰 Weight를 갖고 있음을 앞서 확인했는데, 이는 4종류의 서로 다른 Weight를 가진 RNN이라고 이해하는 것이 좋다. 
- 각 Weight들은 Gate라는 구조에 포함되어 어떤 정보를 기억하고, 어떤 정보를 다음 스텝에 전달할지 등을 결정한다. 
- LSTM에는 Cell state라는 새로운 개념이 추가되는데, 긴 문장이 들어와도 Cell state를 통해 오래된 기억 또한 큰 손실 없이 저장해준다. 앞서 언급한 Gate들이 Cell state에 정보를 추가하거나 빼주는 역할을 한다. 

### 양방향(Bidirectional) RNN
- 진행 방향이 반대인 RNN을 2개 겹쳐놓은 형태다. 
- 문장 분석이나 생성보다는 주로 기계번역 같은 태스크에 유리하다. 
- 번역기를 만들 때, 양방향 RNN 계열의 네트워크 혹은 동일한 효과를 내는 Transformer 네트워크를 주로 사용하게 된다. 

In [6]:
import tensorflow as tf

sentence = "What time is it ?"
dic = {
    "is": 0,
    "it": 1,
    "What": 2,
    "time": 3,
    "?": 4
}

sentence_tensor = tf.constant([[dic[word] for word in sentence.split()]])

embedding_layer = tf.keras.layers.Embedding(input_dim=len(dic), output_dim=100)
emb_out = embedding_layer(sentence_tensor)

print("입력 문장 데이터 형태:", emb_out.shape)

bi_rnn = \
tf.keras.layers.Bidirectional(
    tf.keras.layers.SimpleRNN(units=64, use_bias=False, return_sequences=True)
)
bi_out = bi_rnn(emb_out)

print("Bidirectional RNN 결과 (최종 Step Output):", bi_out.shape)

입력 문장 데이터 형태: (1, 5, 100)
Bidirectional RNN 결과 (최종 Step Output): (1, 5, 128)


- Bidirectional RNN은 순방향 Weight와 역방향 Weight를 각각 정의하므로 앞에서 배운 RNN의 2배 크기의 Weight가 정의된다. 